In [1]:
#import necessary modules
import pandas as pd
import numpy as np
import os
import xml.etree.cElementTree as et
import spacy
import nltk
from tqdm.notebook import tqdm
import textacy
import string
import re

nlp = spacy.load("en_core_web_sm")
pd.options.display.max_rows = 10000
pd.set_option('display.max_colwidth', None)

In [2]:
#Set path to home dictionnary
path = '/home/deepika/Documents/eurecom/project3/afp 2021/2021'
#create a list to contain the sentences
sentences_list = []

In [3]:
#Iterate through the files to extract sentences
#Root directory -> Level 1
for dir1 in tqdm(os.listdir(path)):
    #Root directory -> Level 1 -> Level 2
    for dir2 in (os.listdir(os.path.join(path,dir1))):
        #Root directory -> Level 1 -> Level 2 -> Files
        for dir3 in (os.listdir(os.path.join(path, dir1, dir2))):
            #Parse through to extract sentences
            tree=et.parse(os.path.join(path, dir1, dir2, dir3))
            root=tree.getroot()
            #Obtain sentences from object
            for sentence in (root.iter('p')):
                #Eliminate new lines and lines with no spaces
                 if (sentence.text != None):
                    tokens = nltk.sent_tokenize(sentence.text)    
                    for t in tokens:
                        sentence = t.translate(str.maketrans('', '', string.punctuation))
                        sentence = sentence.strip()
                        #Include only lines with spaces
                        if sentence.count(' ')<4:
                            break
                        else:
                            s_list = [sentence, dir1, dir2, dir3]
                            #print(type(s_list))
                            sentences_list.append(s_list)
                 else:
                        break

  0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
#Convert list to dataframe
sentences_df  = pd.DataFrame(data = sentences_list, 
                             columns = ["sentences","dir1", "dir2","file"])

In [5]:
sentences_df

,sentences,dir1,dir2,file
0,Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight,05,26,afp.com-20210526T175128Z-TX-PAR-WPC31.xml
1,The Council of Ministers has adopted a ban on the entry into Polish airspace of aircraft used by carriers from Belarus government spokesman Piotr Muller said on Twitter,05,26,afp.com-20210526T175128Z-TX-PAR-WPC31.xml
2,This is an element of the implementation of the sanctions requested by the prime minister during the European Council,05,26,afp.com-20210526T175128Z-TX-PAR-WPC31.xml
3,Poland shares a long border with Belarus and the ban is expected to have a major effect on flights by Belarusian carriers including the national airline Belavia to western Europe,05,26,afp.com-20210526T175128Z-TX-PAR-WPC31.xml
4,There was confusion earlier on Wednesday when a Belarusian flight on route from Minsk to Barcelona could be seen on flight tracking websites circling near the Polish border for several minutes,05,26,afp.com-20210526T175128Z-TX-PAR-WPC31.xml
...,...,...,...,...
1398777,Unlike traditional media these platforms are not subject to journalistic standards,11,17,afp.com-20211117T154522Z-TX-PAR-BUX73.xml
1398778,There is no obligation to ensure that the content is true for example,11,17,afp.com-20211117T154522Z-TX-PAR-BUX73.xml
1398779,Under Switzerlands direct democracy voters go to the polls four times a year to decide on pressing issues with many relying on online information,11,17,afp.com-20211117T154522Z-TX-PAR-BUX73.xml
1398780,New platforms provide an easy way to spread illegal and harmful content while users have no rights or only insufficient ones in dealing with online platforms said the statement,11,17,afp.com-20211117T154522Z-TX-PAR-BUX73.xml


In [6]:
import numpy as np
lengths =sentences_df["sentences"].str.len()
argmax = np.where(lengths == lengths.max())[0]
s = sentences_df.iloc[argmax]
print(s)

In [7]:
sentences_df['sentences'].str.len().mean()

144.09107637930714

In [8]:
print(len('Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight'))

186


In [9]:
s_list_from_df = sentences_df[0:1000]
verb_sentence_dir = []
verbs_list = []

In [10]:
for i in tqdm(s_list_from_df.index):
    s = s_list_from_df['sentences'][i]
    tokens = nltk.word_tokenize(s)
    pos_tagged_tokens = nltk.pos_tag(tokens)
    list_of_verbs = []
    path_to_file = os.path.join('/home/deepika/Documents/eurecom/project3/afp 2021/2021/',
                                str(s_list_from_df['dir1'][i]),
                                str(s_list_from_df['dir2'][i]), 
                                s_list_from_df['file'][i])
    for k in range(len(pos_tagged_tokens)):
        if pos_tagged_tokens[k][1].startswith('V'):
            list_of_verbs.append(pos_tagged_tokens[k][0])
            verbs_list.append(pos_tagged_tokens[k][0])
    
    verb_sentence_dir.append([s, list_of_verbs, path_to_file])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [11]:
print(verb_sentence_dir)

[['Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight', ['adopted', 'using', 'agreed', 'arrested'], '/home/deepika/Documents/eurecom/project3/afp 2021/2021/05/26/afp.com-20210526T175128Z-TX-PAR-WPC31.xml'], ['The Council of Ministers has adopted a ban on the entry into Polish airspace of aircraft used by carriers from Belarus government spokesman Piotr Muller said on Twitter', ['has', 'adopted', 'used', 'said'], '/home/deepika/Documents/eurecom/project3/afp 2021/2021/05/26/afp.com-20210526T175128Z-TX-PAR-WPC31.xml'], ['This is an element of the implementation of the sanctions requested by the prime minister during the European Council', ['is', 'requested'], '/home/deepika/Documents/eurecom/project3/afp 2021/2021/05/26/afp.com-20210526T175128Z-TX-PAR-WPC31.xml'], ['Poland shares a long border with Belarus and the ban is expected to have a major effect o

In [12]:
verbs_list = [item.lower() for item in verbs_list]

In [13]:
print(len(verbs_list))

3546


In [14]:
from itertools import chain
verb_list = []
for item in tqdm(verbs_list):
    verb_list.append([token.lemma_ for token in nlp(item)])
verb_list = list(chain.from_iterable(verb_list))
print(verb_list, len(verb_list))

  0%|          | 0/3546 [00:00<?, ?it/s]

['adopt', 'use', 'agree', 'arrest', 'have', 'adopt', 'use', 'say', 'be', 'request', 'be', 'expect', 'have', 'include', 'western', 'be', 'be', 'see', 'track', 'circle', 'say', 'have', 'deny', 'have', 'say', 'allow', 'be', 'allow', 'land', 'say', 'allow', 'amount', 'tell', 'be', 'adopt', 'follow', 'be', 'close', 'say', 'agree', 'ban', 'urge', 'eubase', 'fly', 'have', 'say', 'do', 'have', 'impose', 'follow', 'have', 'garner', 'facilitate', 'be', 'hold', 'evolve', 'organize', 'announce', 'deepdive', 'focus', 'allow', 'schedule', 'maximize', 'continue', 'grow', 'join', 'be', 'expect', 'showcase', 'involve', 'attract', 'lead', 'be', 'grow', 'strengthen', 'be', 'livestreame', 'be', 'hold', 'showcase', 'be', 'meet', 'be', 'introduce', 'etc', 'show', 'serve', 'consist', 'regard', 'include', 'cover', 'bring', 'be', 'bio', 'be', 'advance', 'develop', 'introduce', 'host', 'maintain', 'be', 'present', 'trend', 'have', 'discover', 'see', 'be', 'livestreame', 'be', 'serve', 'invigorate', 'have', 'be'

In [15]:
enable_catena_list = [ "aid", "allow","enable", "help","leave", "let", "permit", "empower",
              "facilitate","afford", "provide", "activate","authorize", "authorise", 
              "ensure", "guarantee"]
prevent_catena_list = [ "bar", "block", "constrain", "deter", "discourage","dissuade", "hamper",
                "hinder", "hold", "impede", "prevent", "protect", "restrain", "restrict",
                "deny" ,"obstruct", "inhibit", "prohibit", "forestall", "impede", "avert",
                "avoid"]

In [16]:
from wordhoard import Synonyms

#Obtain a dictionary of verbs
prevent_list_synonyms = []
for p in prevent_catena_list:
    synonym = Synonyms(search_string=p)
    synonym_results = synonym.find_synonyms()
    #print(p, synonym_results)
    prevent_list_synonyms.append(synonym_results)

In [18]:
from wordhoard import Synonyms

#Obtain synonyms for every word
enable_list_synonyms = []
for p in enable_catena_list:
    synonym = Synonyms(search_string=p)
    synonym_results = synonym.find_synonyms()
    enable_list_synonyms.append(synonym_results)

In [19]:
#COnvert 2d list to 1d list
prevent_list_synonyms = list(chain.from_iterable(prevent_list_synonyms))
#print(len(prevent_list_synonyms), prevent_list_synonyms)

prevent_list_synonyms_2 = [x for x in prevent_list_synonyms
                           if prevent_list_synonyms.count(x)>=2]
print(prevent_list_synonyms_2, len(prevent_list_synonyms_2))

[' debar ', ' disallow ', ' exclude ', ' forbid ', ' interdict ', ' nix ', ' proscribe ', 'cake', 'ingot', 'slab', ' bar ', 'bar', 'cake', 'ingot', 'slab', 'stop', ' bound ', ' confine ', ' curb ', ' limit ', ' restrain ', ' restrict ', ' trammel ', 'ban', 'bar', 'bind', 'bottle up', 'bridle', 'chain', 'check', 'confine', 'constrict', 'cork', 'curb', 'disallow', 'hem in', 'hog-tie', 'hold back', 'hold down', 'imprison', 'inhibit', 'jail', 'keep lid on', 'put half nelson on', 'shotgun', 'withhold', ' disapprove ', ' reject ', 'avert', 'block', 'chill', 'cool', 'dampen', 'daunt', 'debar', 'disadvise', 'discourage', 'divert', 'forestall', 'forfend', 'frighten', 'hinder', 'impede', 'intimidate', 'obstruct', 'obviate', 'preclude', 'prevent', 'prohibit', 'put off', 'restrain', 'rule out', 'scare', 'shut out', 'stave off', 'stop', 'throw cold water on', 'warn', ' disapprove ', ' reject ', 'chill', 'dampen', 'daunt', 'deprecate', 'frighten', 'intimidate', 'repress', 'scare', 'throw cold water 

In [20]:
#COnvert 2d list to 1d
enable_list_synonyms = list(chain.from_iterable(enable_list_synonyms))
#print(len(enable_list_synonyms), enable_list_synonyms)

enable_list_synonyms_2 = [x for x in enable_list_synonyms if enable_list_synonyms.count(x)>=2]
print(len(enable_list_synonyms_2), enable_list_synonyms_2)

223 [' alleviate ', ' assist ', ' attend ', ' attend to ', ' avail ', ' back up ', ' benefact ', ' bootstrap ', ' care ', ' ease ', ' expedite ', ' facilitate ', ' give care ', ' hasten ', ' help out ', ' serve ', ' subserve ', ' succor ', ' succour ', ' support ', ' wait on ', 'advice', 'allowance', 'assist', 'assistance', 'benefit', 'care', 'comfort', 'cooperation', 'favor', 'guidance', 'hand', 'help', 'lift', 'service', 'sustenance', ' let ', ' permit ', 'concede', 'grant', ' empower ', ' equip ', 'accredit', 'approve', 'capacitate', 'commission', 'empower', 'fit', 'implement', 'invest', 'license', 'permit', 'prepare', 'qualify', 'ready', 'sanction', 'warrant', ' aid ', ' alleviate ', ' assist ', ' attend ', ' attend to ', ' avail ', ' back up ', ' benefact ', ' bootstrap ', ' care ', ' ease ', ' expedite ', ' facilitate ', ' give care ', ' hasten ', ' help out ', ' serve ', ' subserve ', ' succor ', ' succour ', ' support ', ' wait on ', 'advice', 'aid', 'assist', 'assistance', 'be

In [21]:
#Remove spaces between words and quotes
enable_list_synonyms_2 = ([i.strip() for i in enable_list_synonyms_2])
prevent_list_synonyms_2 = ([i.strip() for i in prevent_list_synonyms_2])

#keep only unique words
x = np.array(verb_list)
verb_list = list(np.unique(x))

x = np.array(prevent_list_synonyms_2)
prevent_list_synonyms_2 = list(np.unique(x))

x = np.array(enable_list_synonyms_2)
enable_list_synonyms_2 = list(np.unique(x))

In [22]:
print(len(verb_list), len(enable_list_synonyms_2), len(prevent_list_synonyms_2))

verb_dict = {key:None for key in verb_list}


823 84 167


In [23]:
#extract prevent verbs from sentences
prevent_list_from_sentence_all = [v for v in verb_list 
                              if v in prevent_list_synonyms_2 or v in prevent_catena_list]

In [24]:
print(prevent_list_from_sentence_all, len(prevent_list_from_sentence_all))

['anticipate', 'arrest', 'avoid', 'ban', 'bar', 'contain', 'control', 'cross', 'curb', 'deflect', 'deny', 'disrupt', 'divert', 'exclude', 'hold', 'impede', 'keep', 'kill', 'limit', 'obstruct', 'oppose', 'preserve', 'prevent', 'protect', 'restrict', 'save', 'slow', 'stay', 'stop', 'warn'] 30


In [25]:
enable_list_from_sentence_all = [v for v in verb_list 
                              if v in enable_list_synonyms_2 or v in enable_catena_list]
print(enable_list_from_sentence_all, len(enable_list_from_sentence_all))

['allow', 'approve', 'attend', 'authorize', 'bear', 'benefit', 'certify', 'clear', 'commission', 'ease', 'empower', 'enable', 'ensure', 'entitle', 'facilitate', 'give', 'grant', 'hand', 'help', 'invest', 'leave', 'let', 'license', 'lift', 'pass', 'provide', 'secure', 'serve', 'service', 'support'] 30


In [26]:
prevent_list_from_sentence_catena_only = [v for v in verb_list 
                              if v in prevent_catena_list]
print(prevent_list_from_sentence_catena_only, len(prevent_list_from_sentence_catena_only))

#extract enable verbs from sentences
enable_list_from_sentence_catena_only = [v for v in verb_list 
                              if  v in enable_catena_list]
print(enable_list_from_sentence_catena_only, len(enable_list_from_sentence_catena_only))

['avoid', 'bar', 'deny', 'hold', 'impede', 'obstruct', 'prevent', 'protect', 'restrict'] 9
['allow', 'authorize', 'empower', 'enable', 'ensure', 'facilitate', 'help', 'leave', 'let', 'provide'] 10


In [27]:
ambiguous_list_from_sentence = [x for x in prevent_list_from_sentence_all 
                                if x in enable_list_from_sentence_all]
print(ambiguous_list_from_sentence)

[]


In [28]:
enable_sentences = {key:None for key in enable_list_from_sentence_all}
prevent_sentences = {key:None for key in prevent_list_from_sentence_all}
print(prevent_sentences, '\n\n', enable_sentences)

{'anticipate': None, 'arrest': None, 'avoid': None, 'ban': None, 'bar': None, 'contain': None, 'control': None, 'cross': None, 'curb': None, 'deflect': None, 'deny': None, 'disrupt': None, 'divert': None, 'exclude': None, 'hold': None, 'impede': None, 'keep': None, 'kill': None, 'limit': None, 'obstruct': None, 'oppose': None, 'preserve': None, 'prevent': None, 'protect': None, 'restrict': None, 'save': None, 'slow': None, 'stay': None, 'stop': None, 'warn': None} 

 {'allow': None, 'approve': None, 'attend': None, 'authorize': None, 'bear': None, 'benefit': None, 'certify': None, 'clear': None, 'commission': None, 'ease': None, 'empower': None, 'enable': None, 'ensure': None, 'entitle': None, 'facilitate': None, 'give': None, 'grant': None, 'hand': None, 'help': None, 'invest': None, 'leave': None, 'let': None, 'license': None, 'lift': None, 'pass': None, 'provide': None, 'secure': None, 'serve': None, 'service': None, 'support': None}


In [29]:
s_list = [" ".join([token.lemma_ for token in nlp(x)])
          for x in tqdm(s_list_from_df['sentences'])]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [30]:
s_list = [item.lower() for item in s_list]


In [31]:
for k in enable_sentences.keys():
    x = []
    for s in range(len(s_list)):
        if k in s_list[s].split():
            index = s_list[s].split().index(k)
            tokens = nltk.word_tokenize(s_list_from_df['sentences'][s])
            pos_tagged_tokens = nltk.pos_tag(tokens)
            if pos_tagged_tokens[index][1].startswith('V'):
                x.append(s_list_from_df['sentences'][s]) 
    enable_sentences[k] = x
print(enable_sentences)

{'allow': ['Polish authorities said they had denied it entry to their airspace because France had said it would not allow it through', 'But a separate flight from Minsk to Warsaw was allowed to land', 'Belarusian foreign ministry spokesman Anatoly Glaz said Frances decision not to allow the plane amounted to air piracy', 'BIO KOREA 2021 Business Forum will focus on a virtual partnering system allowing attendees to schedule virtual meetings to maximize their business development and licensing potential', 'AFL games scheduled in Melbourne for the coming weekend are currently allowed to go ahead with fans at up to 85 per cent capacity but government officials warned public events could yet face fresh restrictions', 'Up to four people will be allowed at a table though contact details will have to be recorded', 'The funds from our recent capital increase and these new innovation loans secure our financial situation for the foreseeable future allowing us to focus confidently on the implement

In [32]:
for k in prevent_sentences.keys():
    x = []
    for s in range(len(s_list)):
        if k in s_list[s].split():
            index = s_list[s].split().index(k)
            tokens = nltk.word_tokenize(s_list_from_df['sentences'][s])
            pos_tagged_tokens = nltk.pos_tag(tokens)
            if pos_tagged_tokens[index][1].startswith('V'):
                x.append(s_list_from_df['sentences'][s]) 
    prevent_sentences[k] = x
print(prevent_sentences)

{'anticipate': ['A headed clearance at a corner dropped to Rashford whose dragged volley ricocheted into the path of Cavani to stab home with Rulli unable to recover after anticipating the initial shot', 'A headed clearance at a corner dropped to Rashford whose dragged volley ricocheted into the path of Cavani to stab home with Rulli unable to recover after anticipating the initial shot'], 'arrest': ['Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight', 'On Sunday Belarusian authorities diverted a European plane to arrest an exiled dissident sparking global fury', 'Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight', 'The army and police deployed hundreds of heavily armed troops in the streets of Bujumbura and arrested

In [34]:
if 'adopt' in 'adopted adopt'.split():
    print('yes')
    print('adopted adopt'.index('adopt'))
    tokens = nltk.word_tokenize('adopted adopt')
    pos_tagged_tokens = nltk.pos_tag(tokens)
    print(pos_tagged_tokens[0])

yes
0
('adopted', 'VBN')


In [35]:
prevent_count = 0
catena_p_count = 0
wordnet_p_count = 0
p_word_count = {}
for k in prevent_sentences.keys():
    prevent_count += len(prevent_sentences[k])
    p_word_count[k] = len(prevent_sentences[k])
    if k in prevent_catena_list:
        catena_p_count +=len(prevent_sentences[k])
    elif k in prevent_list_synonyms_2:
        wordnet_p_count +=len(prevent_sentences[k])
print(prevent_count)
print(catena_p_count)
print(wordnet_p_count)
print(p_word_count)

127
45
82
{'anticipate': 2, 'arrest': 14, 'avoid': 6, 'ban': 4, 'bar': 1, 'contain': 7, 'control': 3, 'cross': 4, 'curb': 2, 'deflect': 2, 'deny': 3, 'disrupt': 1, 'divert': 7, 'exclude': 1, 'hold': 20, 'impede': 2, 'keep': 3, 'kill': 6, 'limit': 3, 'obstruct': 1, 'oppose': 1, 'preserve': 1, 'prevent': 6, 'protect': 5, 'restrict': 1, 'save': 3, 'slow': 2, 'stay': 1, 'stop': 3, 'warn': 12}


In [36]:
enable_count = 0
catena_e_count = 0
wordnet_e_count = 0
e_word_count = {}
for k in enable_sentences.keys():
    enable_count += len(enable_sentences[k])
    e_word_count[k] = len(enable_sentences[k])
    if k in enable_catena_list:
        catena_e_count +=len(enable_sentences[k])
    elif k in enable_list_synonyms_2:
        wordnet_e_count +=len(enable_sentences[k])
print(enable_count)
print(catena_e_count)
print(wordnet_e_count)
print(e_word_count)

145
60
85
{'allow': 7, 'approve': 4, 'attend': 6, 'authorize': 1, 'bear': 13, 'benefit': 6, 'certify': 1, 'clear': 1, 'commission': 1, 'ease': 1, 'empower': 3, 'enable': 6, 'ensure': 2, 'entitle': 0, 'facilitate': 1, 'give': 16, 'grant': 1, 'hand': 5, 'help': 13, 'invest': 2, 'leave': 8, 'let': 1, 'license': 2, 'lift': 3, 'pass': 2, 'provide': 18, 'secure': 3, 'serve': 12, 'service': 1, 'support': 5}


In [41]:
list_p = []
i = 0
for k in prevent_sentences.keys():
           try:
                list_p.append([k, prevent_sentences[k][0]])
           except:
                continue


In [42]:
print(len(list_p), list_p)

30 [['anticipate', 'A headed clearance at a corner dropped to Rashford whose dragged volley ricocheted into the path of Cavani to stab home with Rulli unable to recover after anticipating the initial shot'], ['arrest', 'Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight'], ['avoid', 'The EU banned Belarusian planes from the blocs airspace and urged EU airlines to avoid flying over the exSoviet country ruled by Alexander Lukashenko often dubbed Europes last dictator'], ['ban', 'EU leaders at their summit this week agreed in principle to banning Belarusian airlines from the bloc and urged EUbased carriers not to fly through Belarusian airspace'], ['bar', 'Lausanne takes place a couple of weeks after the Tokyo Olympics where overseas fans are already barred with organisers set to decide in June how many domestic spectators can attend if any at all'], ['co

In [45]:
list_e = []
i = 0
for k in enable_sentences.keys():
           try:
                list_e.append([k, enable_sentences[k][0]])
           except:
                continue



In [46]:
print(len(list_e), list_e)

29 [['allow', 'Polish authorities said they had denied it entry to their airspace because France had said it would not allow it through'], ['approve', 'However in no way does this constitute a guarantee of future performance and these projections may be reconsidered based on changes in economic conditions and financial markets as well as a certain number of risks and uncertainties including those described in the Valbiotis registration document approved by the French Financial Markets Regulator AMF on 31 July 2020 application number R 20018'], ['attend', 'Jansa who only briefly attended the impeachment debate argued that his government had been succesful in dealing with the health and economic crisis'], ['authorize', 'Bryan Garnier  Co is a fully registered broker dealer authorized and regulated by the FCA in the UK the AMF in Europe and the FINRA in the US Bryan Garnier  Co is headquartered in London with additional offices in Paris Munich Stockholm Oslo Reykjavik and New York'], ['be

In [47]:
list_ee = [[list_e[i][1], list_e[i][0]] for i in range(len(list_e)) ]

In [48]:
enable_work_dataframe = pd.DataFrame(columns=['sentence', 'verb'],
                                    data = list_ee)
enable_work_dataframe['sentence'][0]

'Polish authorities said they had denied it entry to their airspace because France had said it would not allow it through'

In [49]:
enable_work_dataframe['trigger_1'] = ""
enable_work_dataframe['trigger_2'] = ""

In [50]:
enable_work_dataframe.to_csv('/home/deepika/Documents/eurecom/project3/afp 2021/scripts/enable_df.csv', 
                    sep=',', index=True)

In [51]:
list_pp = [[list_p[i][1], list_p[i][0]] for i in range(len(list_p)) ]
print(list_pp)

[['A headed clearance at a corner dropped to Rashford whose dragged volley ricocheted into the path of Cavani to stab home with Rulli unable to recover after anticipating the initial shot', 'anticipate'], ['Poland on Wednesday adopted a ban on Belarusian flights using its airspace as part of EUwide sanctions agreed this week after Belarus arrested a dissident journalist on a diverted flight', 'arrest'], ['The EU banned Belarusian planes from the blocs airspace and urged EU airlines to avoid flying over the exSoviet country ruled by Alexander Lukashenko often dubbed Europes last dictator', 'avoid'], ['EU leaders at their summit this week agreed in principle to banning Belarusian airlines from the bloc and urged EUbased carriers not to fly through Belarusian airspace', 'ban'], ['Lausanne takes place a couple of weeks after the Tokyo Olympics where overseas fans are already barred with organisers set to decide in June how many domestic spectators can attend if any at all', 'bar'], ['Dismi

In [52]:
p_work_dataframe = pd.DataFrame(columns=['sentence', 'verb'],
                                    data = list_pp)
p_work_dataframe['trigger_1'] = ""
p_work_dataframe['trigger_2'] = ""
p_work_dataframe.to_csv('/home/deepika/Documents/eurecom/project3/afp 2021/scripts/prevent_df.csv', 
                    sep=',', index=True)